# AILS Comprehensive Experiments

This notebook runs comprehensive experiments for the AILS paper:
- 100+ scenarios per benchmark map
- Multiple grid sizes (50-500)
- Multiple obstacle patterns (open, maze, uniform, clustered)
- Comparison: Standard A*, AILS-Base, AILS-Adaptive

**Author:** Amr Elshahed  
**Institution:** Universiti Sains Malaysia

In [ ]:
import sys
sys.path.insert(0, '.')

import numpy as np
import pandas as pd
from tqdm import tqdm
import time
from datetime import datetime
import os

from AILS_complete import (
    AILSPathfinder, AILSConfig,
    GridGenerator, MovingAIMapLoader,
    run_benchmark, compute_statistics
)

# Create output directories
os.makedirs('../data/results', exist_ok=True)

print("AILS Experiments Notebook")
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Experiment Configuration

In [ ]:
# Experiment parameters
GRID_SIZES = [50, 100, 200, 300, 400, 500]
PATTERNS = ['open', 'maze', 'uniform', 'clustered']
DENSITIES = [0.1, 0.2, 0.3, 0.4]
NUM_PAIRS_PER_CONFIG = 100
RANDOM_SEED = 42

print(f"Grid Sizes: {GRID_SIZES}")
print(f"Patterns: {PATTERNS}")
print(f"Densities: {DENSITIES}")
print(f"Pairs per config: {NUM_PAIRS_PER_CONFIG}")
print(f"Total configurations: {len(GRID_SIZES) * len(PATTERNS) * len(DENSITIES)}")
print(f"Total path queries: {len(GRID_SIZES) * len(PATTERNS) * len(DENSITIES) * NUM_PAIRS_PER_CONFIG * 3}")

## 2. Grid Generation Functions

In [ ]:
def generate_grid(size, pattern, density, seed):
    """Generate grid based on pattern type."""
    np.random.seed(seed)
    
    if pattern == 'open':
        return GridGenerator.generate_random(size, density * 0.5, seed)
    elif pattern == 'uniform':
        return GridGenerator.generate_random(size, density, seed)
    elif pattern == 'clustered':
        return GridGenerator.generate_clustered(size, density, max(5, size // 20), seed)
    elif pattern == 'maze':
        return GridGenerator.generate_maze(size, seed)
    else:
        raise ValueError(f"Unknown pattern: {pattern}")

# Test grid generation
test_grid = generate_grid(100, 'uniform', 0.3, 42)
print(f"Test grid shape: {test_grid.shape}")
print(f"Obstacle ratio: {np.mean(test_grid):.2%}")

## 3. Run Synthetic Grid Experiments

In [ ]:
def run_synthetic_experiments():
    """Run experiments on synthetic grids."""
    all_results = []
    
    total_configs = len(GRID_SIZES) * len(PATTERNS) * len(DENSITIES)
    pbar = tqdm(total=total_configs, desc="Configurations")
    
    for size in GRID_SIZES:
        for pattern in PATTERNS:
            for density in DENSITIES:
                # Skip invalid combinations
                if pattern == 'maze' and density > 0.1:
                    pbar.update(1)
                    continue
                
                try:
                    # Generate grid
                    grid = generate_grid(size, pattern, density, RANDOM_SEED)
                    
                    # Run benchmark
                    results = run_benchmark(grid, NUM_PAIRS_PER_CONFIG, RANDOM_SEED)
                    
                    # Process results
                    for method, result_list in results.items():
                        for i, r in enumerate(result_list):
                            all_results.append({
                                'grid_size': size,
                                'pattern': pattern,
                                'density': density,
                                'method': method,
                                'pair_id': i,
                                'time_ms': r.time_ms,
                                'nodes_visited': r.nodes_visited,
                                'cost': r.cost,
                                'path_found': r.path_found,
                                'corridor_size': r.corridor_size,
                                'iterations': r.iterations
                            })
                            
                except Exception as e:
                    print(f"Error at {size}x{size} {pattern} d={density}: {e}")
                
                pbar.update(1)
    
    pbar.close()
    return pd.DataFrame(all_results)

# Run experiments (this may take a while)
print("Running synthetic grid experiments...")
start_time = time.time()
df_synthetic = run_synthetic_experiments()
elapsed = time.time() - start_time
print(f"\nCompleted in {elapsed/60:.1f} minutes")
print(f"Total results: {len(df_synthetic)}")

## 4. Run Moving AI Benchmark Experiments

In [ ]:
def run_movingai_experiments():
    """Run experiments on Moving AI Lab benchmark maps."""
    benchmark_maps = [
        '../data/benchmark_maps/den312d.map',
        '../data/benchmark_maps/ht_chantry.map',
        '../data/benchmark_maps/random-64-64-20.map'
    ]
    
    all_results = []
    
    for map_path in benchmark_maps:
        map_name = os.path.basename(map_path).replace('.map', '')
        
        if not os.path.exists(map_path):
            print(f"Map not found: {map_path}")
            continue
        
        print(f"Processing {map_name}...")
        
        try:
            # Load map
            grid = MovingAIMapLoader.load_map(map_path)
            print(f"  Grid size: {grid.shape}")
            
            # Run benchmark
            results = run_benchmark(grid, NUM_PAIRS_PER_CONFIG, RANDOM_SEED)
            
            # Process results
            for method, result_list in results.items():
                for i, r in enumerate(result_list):
                    all_results.append({
                        'map_name': map_name,
                        'grid_size': max(grid.shape),
                        'method': method,
                        'pair_id': i,
                        'time_ms': r.time_ms,
                        'nodes_visited': r.nodes_visited,
                        'cost': r.cost,
                        'path_found': r.path_found,
                        'corridor_size': r.corridor_size,
                        'iterations': r.iterations
                    })
                    
        except Exception as e:
            print(f"  Error: {e}")
    
    return pd.DataFrame(all_results)

# Run Moving AI experiments
print("\nRunning Moving AI benchmark experiments...")
df_movingai = run_movingai_experiments()
print(f"Total Moving AI results: {len(df_movingai)}")

## 5. Results Summary

In [ ]:
def summarize_results(df, group_cols):
    """Generate summary statistics."""
    summary = df.groupby(group_cols + ['method']).agg({
        'time_ms': ['mean', 'std', 'median'],
        'nodes_visited': ['mean', 'std', 'median'],
        'path_found': 'mean',
        'corridor_size': 'mean'
    }).round(3)
    
    summary.columns = ['_'.join(col).strip() for col in summary.columns]
    return summary.reset_index()

# Synthetic results summary
if len(df_synthetic) > 0:
    print("\n" + "="*70)
    print("SYNTHETIC GRID RESULTS")
    print("="*70)
    
    summary_synthetic = summarize_results(df_synthetic, ['grid_size', 'pattern'])
    print(summary_synthetic.to_string())

# Moving AI results summary  
if len(df_movingai) > 0:
    print("\n" + "="*70)
    print("MOVING AI BENCHMARK RESULTS")
    print("="*70)
    
    summary_movingai = summarize_results(df_movingai, ['map_name'])
    print(summary_movingai.to_string())

## 6. Save Results

In [ ]:
# Save to CSV
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

if len(df_synthetic) > 0:
    synthetic_path = f'../data/results/synthetic_results_{timestamp}.csv'
    df_synthetic.to_csv(synthetic_path, index=False)
    print(f"Saved: {synthetic_path}")

if len(df_movingai) > 0:
    movingai_path = f'../data/results/movingai_results_{timestamp}.csv'
    df_movingai.to_csv(movingai_path, index=False)
    print(f"Saved: {movingai_path}")

print("\nExperiments complete!")

## 7. Quick Visualization

In [ ]:
import matplotlib.pyplot as plt

if len(df_synthetic) > 0:
    # Filter successful paths
    df_success = df_synthetic[df_synthetic['path_found']]
    
    # Plot time vs grid size
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Time comparison
    for method in df_success['method'].unique():
        df_m = df_success[df_success['method'] == method]
        means = df_m.groupby('grid_size')['time_ms'].mean()
        axes[0].plot(means.index, means.values, marker='o', label=method)
    
    axes[0].set_xlabel('Grid Size')
    axes[0].set_ylabel('Time (ms)')
    axes[0].set_title('Search Time vs Grid Size')
    axes[0].legend()
    axes[0].set_yscale('log')
    
    # Nodes comparison
    for method in df_success['method'].unique():
        df_m = df_success[df_success['method'] == method]
        means = df_m.groupby('grid_size')['nodes_visited'].mean()
        axes[1].plot(means.index, means.values, marker='o', label=method)
    
    axes[1].set_xlabel('Grid Size')
    axes[1].set_ylabel('Nodes Visited')
    axes[1].set_title('Nodes Visited vs Grid Size')
    axes[1].legend()
    axes[1].set_yscale('log')
    
    plt.tight_layout()
    plt.savefig('../data/results/quick_comparison.png', dpi=150)
    plt.show()
    
print("Visualization complete!")